# Feature engineering

* Feature selection
* Feature transformation

In [134]:
import pandas as pd

In [135]:
df = pd.read_csv('../input/cleaned_data.csv')

df.drop('Unnamed: 0', axis='columns', inplace=True)

df

,building_id,building_type,primary_property_type,property_name,neighborhood,list_of_all_property_use_types,largest_property_use_type,second_largest_property_use_type,default_data,compliance_status,...,site_energy_use_target,site_energy_use_wn,steam_use,electricity_kwh,electricity,natural_gas_therms,natural_gas,emissions_target,emissions_intensity,is_agregation
0,23163,NonResidential,Medical Office,LESCHI CENTER,DOWNTOWN,Medical Office,Medical Office,NaN,Yes,Compliant,...,1.798899e+06,1.798899e+06,0.000,5.272270e+05,1.798974e+06,0.000000,0.000000e+00,12.540,0.110,False
1,24408,SPS-District K-12,K-12 School,WILSON-PACIFIC (SPS-DISTRICT),NORTHWEST,K-12 School,K-12 School,NaN,Yes,Compliant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,23848,NonResidential,Restaurant\n,SPAGHETTI FACTORY,DOWNTOWN,"Office, Restaurant",Restaurant,Office,No,Compliant,...,3.816695e+06,3.688427e+06,0.000,5.316360e+05,1.814019e+06,20028.000000,2.002751e+06,119.010,2.760,False
3,21471,NonResidential,Other,EITEL BUILDING (2ND AND PIKE BUILDING),DOWNTOWN,"Other, Personal Services (Health/Beauty, Dry C...",Other,Retail Store,No,Compliant,...,4.139190e+05,4.139190e+05,0.000,1.115070e+05,3.804780e+05,335.000000,3.345700e+04,4.430,0.060,False
4,21237,NonResidential,Retail Store,STAPLES,NORTHWEST,"Parking, Retail Store",Retail Store,Parking,Yes,Compliant,...,1.392153e+06,1.477460e+06,0.000,2.466200e+05,8.415010e+05,5507.000000,5.506860e+05,35.110,1.330,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1693,775,NonResidential,Large Office,Columbia Center - 2015,DOWNTOWN,"Office, Parking",Office,Parking,False,Compliant,...,9.374886e+07,9.289635e+07,0.000,2.617277e+07,8.930337e+07,44473.816405,4.447364e+06,858.760,0.340,True
1694,753,NonResidential,Other,WestinBuilding,DOWNTOWN,"Data Center, Office",Data Center,Office,False,Compliant,...,2.797747e+08,2.797966e+08,0.000,8.195296e+07,2.796294e+08,1511.360107,1.511555e+05,1957.390,3.135,True
1695,752,NonResidential,Large Office,ACTIVE VOICE,DOWNTOWN,"Bar/Nightclub, Office, Personal Services (Heal...",Office,Bar/Nightclub,False,Compliant,...,1.079566e+07,1.143229e+07,2806907.625,1.945288e+06,6.637447e+06,13514.160155,1.351432e+06,334.705,1.905,True
1696,857,NonResidential,Warehouse,3480 W Marginal Way SW,SOUTHWEST,"Non-Refrigerated Warehouse, Office",Non-Refrigerated Warehouse,Office,False,Compliant,...,1.156778e+06,1.182483e+06,0.000,2.852932e+05,9.734410e+05,1833.715027,1.833580e+05,16.525,0.095,True


## Correlation between the features

We must remove variables that are too much correlated to our targets :

* `emissions_target`
* `site_energy_use_target`

In [136]:
# https://github.com/JamesIgoe/GoogleFitAnalysis/blob/master/Analysis.ipynb

def corrFilter(x: pd.DataFrame, bound: float):
    xCorr = x.corr()
    xFiltered = xCorr[((xCorr >= bound) | (xCorr <= -bound)) & (xCorr !=1.000)]
    return xFiltered

def corrFilterFlattened(x: pd.DataFrame, bound: float):
    xFiltered = corrFilter(x, bound)
    xFlattened = xFiltered.unstack().sort_values().drop_duplicates()
    return xFlattened

def filterForLabels(df: pd.DataFrame, label):
    df = df.sort_index()
    try:
        sideLeft = df[label,]
    except:
        sideLeft = pd.DataFrame()

    try:
        sideRight = df[:,label]
    except:
        sideRight = pd.DataFrame()

    if sideLeft.empty and sideRight.empty:
        return pd.DataFrame()
    elif sideLeft.empty:        
        concat = sideRight.to_frame(name='correlation').rename_axis('variable').reset_index(level=0)
        return concat
    elif sideRight.empty:
        concat = sideLeft.to_frame(name='correlation').rename_axis('variable').reset_index(level=0)
        return concat
    else:
        concat = pd.concat([sideLeft,sideRight], axis=1)
        concat['correlation'] = concat[0].fillna(0) + concat[1].fillna(0)
        concat.drop(columns=[0,1], inplace=True)

        return concat.rename_axis('variable').reset_index(level=0)

In [137]:
corr_df = corrFilterFlattened(df, 0.7)


filterForLabels(corr_df, 'emissions_target')

,variable,correlation
0,natural_gas,0.737067
1,natural_gas_therms,0.737067
2,site_energy_use_target,0.861910
3,site_energy_use_wn,0.859331


In [138]:
variables_to_remove = filterForLabels(corr_df, 'site_energy_use_target')['variable'].tolist()
df = df.drop(variables_to_remove, axis='columns')

In [139]:
# remove useless variables

df = df.drop([
        'building_id',
        'property_name',
        'default_data',
        'compliance_status',
        'site_eui',
        'site_euiwn',
        'source_euiwn',
        'source_eui',
        'steam_use',
        'emissions_intensity',
        'natural_gas',
        'natural_gas_therms',
        'second_largest_property_use_type_gfa',
        'second_largest_property_use_type',
        'latitude',
        'longitude',
        'address',
        'data_year',
        'is_agregation',
        'zip_code',
        ],axis='columns')

In [140]:
corr = df.corr()
corr.style.background_gradient(cmap='RdBu', vmin = -1, vmax = 1)

,zip_code,council_district_code,year_built,number_of_floors,property_gfa_parking,energystar_score,site_eui,site_energy_use_target
zip_code,1.000000,-0.196192,0.101942,-0.144932,-0.100441,0.010473,-0.064862,-0.050810
council_district_code,-0.196192,1.000000,-0.051464,0.274422,0.200807,0.083467,0.089137,0.052812
year_built,0.101942,-0.051464,1.000000,0.095521,0.289599,-0.031097,0.127640,0.070705
number_of_floors,-0.144932,0.274422,0.095521,1.000000,0.505556,0.115653,0.025231,0.220255
property_gfa_parking,-0.100441,0.200807,0.289599,0.505556,1.000000,0.104035,0.065262,0.183019
energystar_score,0.010473,0.083467,-0.031097,0.115653,0.104035,1.000000,-0.352225,-0.078893
site_eui,-0.064862,0.089137,0.127640,0.025231,0.065262,-0.352225,1.000000,0.273301
site_energy_use_target,-0.050810,0.052812,0.070705,0.220255,0.183019,-0.078893,0.273301,1.000000


In [141]:
print('Done')

Done
